In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter
from random import choice
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model
from scipy.optimize import curve_fit

%matplotlib widget
trajectories = pd.read_csv('211004_ExNo_72_FM4-64_60x__df_total.csv')


In [2]:
molecules_tot = trajectories['Molecule#'].unique()

In [8]:
trajectories

,timepoint,frame,time[s],x_position[nm],y_position[nm],x_displacement[nm],y_displacement[nm],Euclidean_p2p_distance[nm],Speed[um/s],Velocity[um/s],...,Cumulative_Euclidean_distance_2D,Cumulative_distance_along_spline_1D,Orthogonal_distance_to_spline,Transposed_euclidean_dist_p2p[nm],Displ_along_spline_p2p[nm],Ort_displacements_p2p,P2P_Velocities_Cumulative_Euclidean_distance,P2P_Velocities_Distance_along_spline,Dataset,Molecule#
0,0,1.0,0.000000,10787.821934,126789.966297,0.000000,0.000000,NaN,NaN,NaN,...,0.000000,245891.049020,22086.417969,NaN,345.450386,19.395609,1613.467943,1658.626241,211004_ExNo_72_FM4-64_60x_,0
1,1,2.0,0.050000,10806.844655,126597.301074,-19.022721,192.665223,193.602045,3.872041,3.872041,...,69.716255,246236.499405,22093.402344,-69.716255,0.000000,81.525561,2544.448957,2320.884056,211004_ExNo_72_FM4-64_60x_,0
2,2,3.0,0.100000,10780.758076,126311.530418,26.086579,285.770656,286.958843,5.739177,5.739177,...,172.303940,246236.499405,22122.759766,-102.587685,345.330694,-140.173414,2382.670485,2414.783562,211004_ExNo_72_FM4-64_60x_,0
3,3,4.0,0.150000,10988.611373,125987.858051,-207.853297,323.672367,384.664522,7.693290,7.693290,...,297.752228,246581.830100,22072.283203,-125.448288,690.312134,258.391172,2710.829586,2445.288547,211004_ExNo_72_FM4-64_60x_,0
4,4,5.0,0.200000,10842.389713,125403.353789,146.221660,584.504262,602.516395,12.050328,12.050328,...,499.706818,247272.142234,22165.330078,-201.954590,344.986346,43.786564,2888.454491,2760.302730,211004_ExNo_72_FM4-64_60x_,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58475,62,2413.0,120.599998,21296.806220,84942.328193,-16.043684,348.985373,349.353961,6.987079,6.987079,...,8514.083984,289618.186087,21365.695312,-124.472656,349.757065,-162.758234,2376.246954,2597.078513,211004_ExNo_72_FM4-64_60x_,999
58476,63,2414.0,120.650002,21521.434062,84633.050666,-224.627842,309.277527,382.243713,7.644874,7.644874,...,8638.948242,289967.943151,21307.085938,-124.864258,700.089578,-174.310988,1212.497854,1331.556703,211004_ExNo_72_FM4-64_60x_,999
58477,64,2416.0,120.750000,21810.788513,84044.489381,-289.354451,588.561285,655.843262,13.116865,13.116865,...,8867.402344,290668.032730,21244.316406,-228.454102,0.000000,-7.745227,2986.511728,2856.857780,211004_ExNo_72_FM4-64_60x_,999
58478,65,2417.0,120.800003,21840.733468,83927.546223,-29.944955,116.943158,120.716207,2.414324,2.414324,...,8910.825195,290668.032730,21241.527344,-43.422852,350.337522,38.254260,2429.786256,1682.245107,211004_ExNo_72_FM4-64_60x_,999


In [3]:
size_min = 25
long_traj = [molecule for molecule in molecules_tot if len(trajectories.loc[trajectories['Molecule#']==molecule])>=size_min]
def func(x, D,alpha):
    return(D*x**alpha)

In [4]:
molecule = choice(long_traj)
to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
Y = to_plot['y_position[nm]']
X = to_plot['x_position[nm]']
X = np.array(X).reshape(-1,1)
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
displacement = np.transpose((np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2))
msd = []
for tau in range(size_min//2):
    sd = []
    for index in range(len(displacement)-tau):
        sd.append((displacement[index]-displacement[index+tau])**2)
    msd.append(np.mean(sd))

In [7]:
fig, ax = plt.subplots()

alphas,Ds = [],[]
for i in range(20):
    molecule = choice(long_traj)
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
    Y = to_plot['y_position[nm]']
    X = to_plot['x_position[nm]']
    X = np.array(X).reshape(-1,1)
    lm = linear_model.LinearRegression()
    model = lm.fit(X,Y)
    displacement = np.transpose((np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2))
    msd = []
    for tau in range(size_min//2):
        sd = []
        for index in range(len(displacement)-tau):
            sd.append((displacement[index]-displacement[index+tau])**2)
        msd.append(np.mean(sd))
    taus = [tau*0.05 for tau in range(size_min//2)]
    popt, pcov = curve_fit(func, taus, msd,bounds = ([0,0],2*[np.inf]),p0=(0.1,0.1))
    alphas.append(popt[1])
    Ds.append(popt[0])
    ax.plot(msd,alpha = 0.1)
ax.set_ylim((0,400**2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 160000.0)

In [15]:
fig, ax = plt.subplots()
window = 20
def func(x, D,alpha):
    return(D*x**alpha)
alphas,Ds,molecules,speeds,mean_path_widths = [],[],[],[],[]
for molecule in long_traj:
#     molecule = choice(long_traj)
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
    Y = to_plot['y_position[nm]']
    X = to_plot['x_position[nm]']
    X = np.array(X).reshape(-1,1)
    lm = linear_model.LinearRegression()
    model = lm.fit(X,Y)
    displacement = np.transpose((np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2))
    msd = []
    for tau in range(size_min//2):
        sd = []
        for index in range(len(displacement)-tau):
            sd.append((displacement[index]-displacement[index+tau])**2)
        msd.append(np.mean(sd))
    path_widths = []
    for index in range(len(displacement)-window):
        path_widths.append(np.max(displacement[index:index+tau])-np.min(displacement[index:index+tau]))
    mean_path_widths.append(np.mean(path_widths))
    taus = [tau*0.05 for tau in range(size_min//2)]
    popt, pcov = curve_fit(func, taus, msd,bounds = ([0,0],2*[np.inf]),p0=(0.1,0.1))
    alphas.append(popt[1])
    Ds.append(popt[0])
    molecules.append(molecule)
    speeds.append(np.mean(to_plot['Velocity[um/s]']))
    ax.plot(msd,alpha = 0.1)
ax.set_ylim((0,400**2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 160000.0)

In [13]:
len(taus),len(msd)

(12, 13)

In [20]:
import seaborn as sns
fig, ax = plt.subplots()

sns.distplot(alphas)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:ylabel='Density'>

In [83]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.scatter(np.abs(speeds),alphas,alpha = 0.2)
# ax.set_ylim((0,1e6))
# ax2.hist(np.abs(speeds),20,alpha = 0.2)
sns.distplot(np.abs(speeds),ax=ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:label='7ea4a662-a82b-4f7d-bb5e-11ae09018137', ylabel='Density'>

In [16]:
data = np.array((alphas,Ds,molecules,speeds,mean_path_widths)).transpose()
table_traj = pd.DataFrame(data,columns=("alpha","D","molecule","speed","path_width"))
table_traj = table_traj.loc[table_traj['D']<=1e6]

In [17]:
table_traj['alpha_round'] = (table_traj['alpha']*10).astype(int)/10
table_traj['speed_round'] = (table_traj['speed']).astype(int)
table_traj['abs_speed_round'] = np.abs(table_traj['speed_round'])

In [18]:
y = 'path_width'
data = table_traj.groupby("abs_speed_round")[y]

In [21]:
fig, ax = plt.subplots()
ax.scatter(np.abs(table_traj['speed']),table_traj[y],alpha = 0.2)
ax2 = ax.twinx()

ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()))
ax.set_ylabel(y)
ax.set_xlabel('speed')
# ax.set_ylim((0,0.6e6))
sns.distplot(np.abs(speeds),ax=ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:label='f0d73851-66a9-4ae2-87c7-f312a54b2d85', ylabel='Density'>

In [59]:
from pandas.plotting import scatter_matrix
scatter_matrix(table_traj[["alpha_round","D","speed"]], diagonal="kde",)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<AxesSubplot:xlabel='alpha_round', ylabel='alpha_round'>,
        <AxesSubplot:xlabel='D', ylabel='alpha_round'>,
        <AxesSubplot:xlabel='speed', ylabel='alpha_round'>],
       [<AxesSubplot:xlabel='alpha_round', ylabel='D'>,
        <AxesSubplot:xlabel='D', ylabel='D'>,
        <AxesSubplot:xlabel='speed', ylabel='D'>],
       [<AxesSubplot:xlabel='alpha_round', ylabel='speed'>,
        <AxesSubplot:xlabel='D', ylabel='speed'>,
        <AxesSubplot:xlabel='speed', ylabel='speed'>]], dtype=object)